In [1]:
from keras.applications.resnet50 import ResNet50

# define ResNet50 model
ResNet50_model = ResNet50(weights='imagenet')

Using TensorFlow backend.


In [34]:
import pandas as pd
# load dog breed
df_dog_breed = pd.read_csv('dog_name.csv')
dog_names = df_dog_breed.values

In [19]:
from keras.preprocessing import image  
import urllib.request
from tqdm import tqdm
import numpy as np

def path_to_tensor(img_path, is_url=False):
    if is_url == True:
        urllib.request.urlretrieve(img_path, 'img_buf.jpg')
        img_path = 'img_buf.jpg'
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert image to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    
    if is_url == True:
        os.remove('img_buf.jpg')
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3)
    return np.expand_dims(x, axis=0)


import os

from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True   

In [38]:
from keras.applications.resnet50 import preprocess_input, decode_predictions

def ResNet50_predict_labels(img_path, is_url=False):
    # returns prediction vector for image located at img_path
    img = preprocess_input(path_to_tensor(img_path, is_url))
    return np.argmax(ResNet50_model.predict(img))

### if a dog is detected in the image which is stored at img_path
def dog_detector(img_path, is_url=False):
    prediction = ResNet50_predict_labels(img_path, is_url)
    # 151 to 269 are dogs in ImageNet, if the predicted label is in this range
    # return True
    return ((prediction <= 268) & (prediction >= 151)) 

In [41]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

def load_model():
    model = Sequential()
    model.add(Dense(1024, activation='relu', input_shape=(6144,)))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(133, activation='softmax'))
    model.load_weights('saved_models/weights.best.model1.hdf5')
    return model

model = load_model()


from extract_bottleneck_features import *

def model_predict_breed(model, img_path, is_url=False):
    # extract bottleneck features
    if dog_detector(img_path, is_url=False):
        print('Oh, it\'s a cute dog! Let me check out the breed ...')
    else:
        print('It\s not a dog! Take it away from me!')
        return
        
    keras_tensor = path_to_tensor(img_path, is_url)
    
    print('Let me have a look at the coat ...')
    Resnet50_feature = extract_Resnet50(keras_tensor)
    print('Let me measure the tail length ...')
    InceptionV3_feature = extract_InceptionV3(keras_tensor)
    print('where is the nose? ...')
    Xception_feature = extract_Xception(keras_tensor)
    
    
    
    InceptionV3_flatten_model = Sequential()
    InceptionV3_flatten_model.add(GlobalAveragePooling2D(input_shape=InceptionV3_feature.shape[1:]))
    Xception_flatten_model = Sequential()
    Xception_flatten_model.add(GlobalAveragePooling2D(input_shape=Xception_feature.shape[1:]))
    Resnet50_flatten_model = Sequential()
    Resnet50_flatten_model.add(GlobalAveragePooling2D(input_shape=Resnet50_feature.shape[1:]))
    
    InceptionV3_feature = InceptionV3_flatten_model.predict(InceptionV3_feature)
    Xception_feature = Xception_flatten_model.predict(Xception_feature)
    Resnet50_feature = Resnet50_flatten_model.predict(Resnet50_feature)
    
    feature = np.concatenate((InceptionV3_feature, Xception_feature, Resnet50_feature), axis=1)
    print('OK. I get some information on this dog. Let me look into my Dog Breed Encyclopaedia ...')
    print('It may take awhile ...\n')
    
    # create prediction vactor
    predicted_vector = model.predict(feature)
    # return dog breed that has the maximum likelihood
    print(('According to my encyclopaedia, it\'s a/an {}.').format(dog_names[np.argmax(predicted_vector)][0]))
    return